# Trace - training control and debugging

In this tutorial, we will talk about another important concept in FastEstimator - Trace.

`Trace` is a class contains has 6 event functions below, each event function will be executed on different events of training loop when putting `Trace` inside `Estimator`. If you are a Keras user, you would see that `Trace` is a combination of callbacks and metrics. 
* on_begin
* on_epoch_begin
* on_batch_begin
* on_batch_end
* on_epoch_end
* on_end

`Trace` differs from keras's callback in the following places:
1. Trace has full access to the preprocessing data and prediction data
2. Trace can pass data among each other
3. Trace is simpler and has fewer event functions than keras callbacks

`Trace` can be used for anything that involves training loop, such as changing learning rate, calculating metrics, writing checkpoints and so on.

## debugging training loop with Trace

Since `Trace` can have full access to data used in training loop, one natural usage of `Trace` is debugging training loop, for example, printing network prediction for each batch.

Remember in tutorial 3, we customized an operation that scales the prediction score by 10 and write to a new key, let's see whether the operation is working correctly using `Trace`.

In [ ]:
import tempfile

import numpy as np
import tensorflow as tf

import fastestimator as fe
from fastestimator.architecture import LeNet
from fastestimator.estimator.trace import Accuracy, ModelSaver
from fastestimator.network.loss import SparseCategoricalCrossentropy
from fastestimator.network.model import FEModel, ModelOp
from fastestimator.pipeline.processing import Minmax
from fastestimator.util.op import TensorOp

class Scale(TensorOp):
    def forward(self, data, state):
        data = data * 10
        return data

(x_train, y_train), (x_eval, y_eval) = tf.keras.datasets.mnist.load_data()
train_data = {"x": np.expand_dims(x_train, -1), "y": y_train}
eval_data = {"x": np.expand_dims(x_eval, -1), "y": y_eval}
data = {"train": train_data, "eval": eval_data}
pipeline = fe.Pipeline(batch_size=32, data=data, ops=Minmax(inputs="x", outputs="x"))

# step 2. prepare model
model = FEModel(model_def=LeNet, model_name="lenet", optimizer="adam")
network = fe.Network(
    ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), 
         SparseCategoricalCrossentropy(inputs=("y", "y_pred")), 
         Scale(inputs="y_pred", outputs="y_pred_scaled")])

## define trace

In [2]:
from fastestimator.estimator.trace import Trace

class ShowPred(Trace):
    def on_batch_end(self, state):
        if state["mode"] == "train":
            batch_data = state["batch"]
            print("step: {}".format(state["batch_idx"]))
            print("batch data has following keys: {}".format(list(batch_data.keys())))
            print("scaled_prediction is:")
            print(batch_data["y_pred_scaled"])

# step 3.prepare estimator
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=1, traces=ShowPred(), steps_per_epoch=1)

In [3]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 0; lenet_lr: 0.001; 
step: 0
batch data has following keys: ['y_pred', 'y', 'x', 'loss', 'y_pred_scaled']
scaled_prediction is:
tf.Tensor(
[[1.0597024  0.88230646 0.9054666  1.0526242  1.0112537  1.1514847
  0.9731587  0.9711996  0.84732836 1.1454759 ]
 [1.0177196  0.96111745 0.8916435  1.0738678  0.9751328  1.2481465
  0.9405147  0.87076896 0.8726471  1.148442  ]
 [1.0760062  0.94326234 0.9008551  1.0322686  1.0499443  1.1253775
  0.93624175 0.9271722  